In [22]:
import pytz
import requests
import warnings
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.parse import urljoin
from job_bot import postbot
from api_token import api_key
import pandas as pd

# def init_bot():
bot_token = api_key()

def egujobs(post_jobs=True, jobbot_status=False, current_date=None):
    if jobbot_status:
        sign1 = f"\n----- ** {datetime.now().strftime('%Y-%b-%d')} ** -----"
        sign2 = f"\nEGU Jobs"
        message_text = f" *{'Job_Bot Status: Active'}* {sign1}{sign2}"
        postbot(bot_token, message_text)

    if post_jobs:        
        if current_date is None:
            # Get the current time in the Hawaii timezone
            hawaii_timezone = pytz.timezone('Pacific/Honolulu')
            current_date_hawaii = datetime.now(hawaii_timezone)    
            current_date = current_date_hawaii.strftime("%Y-%m-%d")

        base_url0 = 'https://www.egu.eu/jobs/?'
        base_url1 = 'limit=10&sortby=-created_at&sortby=-created_at&'
        base_url2 = 'page=&keywords=&sector=10&employment_level=30'
        base_url = f'{base_url0}{base_url1}{base_url2}'

        # Send an HTTP GET request to the URL
        response = requests.get(base_url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the elements you want to scrape
            job_list = soup.find_all('div', class_='media-body')

            for job in job_list:
                job_title = job.find('h2').text.strip()
                job_link = urljoin(base_url, job.find('h2').find('a')['href'])

                # Send a request to the job page
                job_response = requests.get(job_link)
                if job_response.status_code == 200:
                    job_soup = BeautifulSoup(job_response.text, 'html.parser')
                    # Location

                    # Find the "Posted" line
                    posted_line = job_soup.find("div",
                                                class_="col-md-3 mb-2 mb-md-0 strong text-md-right",
                                                string='Posted')

                    # Find the date that follows the "Posted" line
                    if posted_line:
                        posted_date = posted_line.find_next_sibling('div').text.strip()
                        input_format = "%d %B %Y"  # day month year (e.g., "29 September 2023")
                        # Parse the posted_date into a datetime object
                        posted_datetime = datetime.strptime(posted_date,
                                                            input_format).strftime('%Y-%m-%d')
                        posted_datetime =  datetime.strptime(posted_datetime, "%Y-%m-%d")
                        converted_date = datetime.strptime(current_date, "%Y-%m-%d")
                        if posted_datetime == converted_date:
                            # print(posted_datetime)
                            location = job_soup.find("div",
                                class_="col-md-3 mb-2 mb-md-0 strong text-md-right",
                                string="Location"
                                )

                            location_content = location.find_next_sibling(
                                "div").text.strip() if location else "Not specified"

                            # Application deadline
                            application_deadline = job_soup.find(
                                "div",
                                class_="col-md-3 mb-2 mb-md-0 strong text-md-right",
                                string="Application deadline")

                            application_deadline_content = application_deadline.find_next_sibling(
                                "div").text.strip() if application_deadline else "Not specified"

                            # Job description
                            job_description = job_soup.find(
                                "div",
                                class_="col-md-3 mb-2 mb-md-0 strong text-md-right",
                                string="Job description")

                            job_description_content = job_description.find_next_sibling(
                                "div").text.strip() if job_description else "Not specified"

                            # How to apply
                            how_to_apply = job_soup.find(
                                "div",
                                class_="col-md-3 mb-2 mb-md-0 strong text-md-right",
                                string="How to apply"
                            )

                            how_to_apply_content = how_to_apply.find_next_sibling(
                                "div").text.strip() if how_to_apply else "Not specified"

                            # # Print the extracted information
                            # print(f"Title: {job_title}")
                            # print(f"Date Posted: {posted_date}")
                            # print(f"Location: {location_content}")
                            # print(f"Deadline: {application_deadline_content}")
                            # print(f"Job Description:\n{job_description_content}")
                            # print(f"How to Apply:\n{how_to_apply_content}")

                            job_heading = f"*{job_title}* \n\nLocation:\
                            {location_content} \nDate Posted: {posted_date} \
                            \nDeadline: {application_deadline_content}"
                            job_desc = f"\n\nDescription:\n{job_description_content} \
                            \n\nHow to Apply:\n{how_to_apply_content}\n--------"

                            full_post = job_heading+job_desc+"---------"

                            postbot(bot_token, full_post)
                            # print(full_post)

In [21]:
egujobs(current_date="2023-11-14")

*Post-doctoral researcher in ocean ensemble modeling* 

Location:                            Grenoble, France 
Date Posted: 14 November 2023                             
Deadline: Open until the position is filled

Description:
Missions:
This post-doctorate concerns the study of the impact of different sources of uncertainty on the physical modeling of the Mediterranean by producing and analysing ensembles of regional NEMO simulations at 1/12°. Once these ensembles have been compared with available spatial and in situ observations using probabilistic scores, they will be used to quantify the relative and cumulative impacts of these uncertainties on the simulated physics and on the amplitude and structure of ensemble dispersions.
The activities of the selected person will concern:
- getting to grips with the stochastic ensemble simulator of the Mediterranean, which is currently being developed on the basis of an existing realistic configuration of the NEMO 1/12° model;
- participating i

In [16]:
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import os
import re
import pytz
from job_bot import postbot
from api_token import api_key

# def init_bot():
bot_token = api_key()

def esjobs(post_jobs=True, jobbot_status=False, current_date=None):
    if jobbot_status:
        sign1 = f"\n----- ** {datetime.now().strftime('%Y-%b-%d')} ** -----"
        sign2 = f"\nEarth Science Jobs"
        message_text = f" *{'Job_Bot Status: Active'}* {sign1}{sign2}"
        postbot(bot_token, message_text)
    
    if post_jobs:
        if current_date is None:
            # Set the time zone to Hawaii Standard Time (HST)
            hawaii_timezone = pytz.timezone('Pacific/Honolulu')

            # Get the current time in the Hawaii timezone
            current_date = datetime.now(hawaii_timezone)    
            print(current_date)
            converted_date = datetime.strptime(current_date, "%Y-%m-%d")
            
        current_time = converted_date.strftime("%B %Y")
        # print(current_time)

        url = "https://mailman.ucar.edu/pipermail/es_jobs_net/"

        # Define user-agent and headers
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; \
            x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,\
            application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "en-US,en;q=0.9,ar;q=0.8",
            "Cache-Control": "max-age=0",
            "Connection": "keep-alive",
            "Host": "mailman.ucar.edu",
            "If-Modified-Since": "Thu, 19 Oct 2023 09:12:44 GMT",
            "If-None-Match": "\"1a8c-6080e289aa9c3\"",
            "Sec-Ch-Ua": "\"Chromium\";v=\"118\", \"Google Chrome\";v=\"118\", \"Not=A?Brand\";v=\"99\"",
            "Sec-Ch-Ua-Mobile": "?0",
            "Sec-Ch-Ua-Platform": "\"macOS\"",
            "Sec-Fetch-Dest": "document",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-Site": "none",
            "Sec-Fetch-User": "?1",
            "Upgrade-Insecure-Requests": "1"
        }

        # Send a request to the URL
        response = requests.get(url, headers=headers)
        page_content = response.text

        soup = BeautifulSoup(page_content, "html.parser")

        # Find the link to the latest thread (October 2023)
        latest_thread_link = None
        for row in soup.find_all("tr"):
            cells = row.find_all("td")
            if cells and current_time in cells[0].get_text():
                latest_thread_link = row.find("a", href=True)["href"]
                break

        tempPath = latest_thread_link.split("/")[0]
        print(tempPath)

        # Construct the URL for the latest thread
        if latest_thread_link:
            latest_thread_url = f"{url.rstrip('/')}/{latest_thread_link}"


            # Send a request to the latest thread's URL and scrape its content
            response_latest_thread = requests.get(latest_thread_url, headers=headers)
            latest_thread_content = response_latest_thread.text

            # Parse and extract data from the latest thread's content using BeautifulSoup
            latest_thread_soup = BeautifulSoup(latest_thread_content, "html.parser")

            # Define keywords to search for
            keywords = ["phd", "ph.d.", "doctoral", "ms ", "m.s.", "master", "assistantship", "fellowship"]

            # Initialize a list to store the links containing the keywords
            filtered_links = []

            # Loop through the links and check if they contain any of the keywords
            for link in latest_thread_soup.find_all("a", href=True):  # Fixed this line
                for keyword in keywords:
                    if re.search(keyword, link.get_text(), re.I):
                        filtered_links.append(link['href'])

            # Print the filtered links
            for link in filtered_links:
        #             print(f"posting {link}")
                # print(os.path.join(url, tempPath, link))
                full_url = os.path.join(url, tempPath, link)
                # Send a request to the filtered link and scrape its content
                response_filtered_link = requests.get(full_url, headers=headers)
                filtered_link_content = response_filtered_link.text

                # Parse and extract data from the filtered link's content using BeautifulSoup
                filtered_link_soup = BeautifulSoup(filtered_link_content, "html.parser")
                # You can perform additional parsing or extraction here, or simply print the content
                # print("Content of filtered link:", full_url)        
        #         print(filtered_link_soup.get_text())  # Print the text content of the filtered link
    #             print("--------------------------------------\n")

                pub_date = filtered_link_soup.find("i").get_text()
                format_str = '%a %b %d %H:%M:%S %Y'
                date_time_str = pub_date[:19]+pub_date[23:]
    #             print(len(date_time_str))
                if len(date_time_str) == 24:
                    date_time_ = datetime.strptime(date_time_str, format_str).strftime("%Y-%m-%d")
                    date_time_ = datetime.strptime(date_time_, "%Y-%m-%d")
                    converted_date = datetime.strptime(current_date, "%Y-%m-%d")
                    if date_time_ == converted_date:
        #                 print(f"found for {date_time_}")


                        # Extract the header
                        header = filtered_link_soup.find("h1")  # Assuming the header is within an h2 element
                        if header:
                            header_text = header.get_text().removeprefix('[ES_JOBS_NET] ')
                            # print("Header:")
                            # print(header_text)

                        # Extract the body
                        body = filtered_link_soup.find("pre")  # Adjust class as needed
                        if body:
                            body_text = body.get_text()
                            # print("\nBody:")
                            split_index = body_text.find("-------------- next part --------------")
                            if split_index != -1:
                            # Keep only the part before "-------------- next part --------------"
                                body_text = body_text[:split_index]
                            # print(body_text)
                        sign1 = f"\n----- ** {datetime.now().strftime('%Y-%b-%d')} ** -----"
                        sign2 = f"\n"
                        message_text = f" *{header_text.title()}* \n\n{body_text}{sign1}{sign2}"
                        # postbot(bot_token, message_text)
                        print(message_text)

In [17]:
esjobs(current_date=None)

2023-11-16 10:08:47.872001-10:00


TypeError: strptime() argument 1 must be str, not datetime.datetime

In [43]:
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import os
import re
from job_bot import postbot
from api_token import api_key
import feedparser
import pytz


bot_token = api_key()

def metjobs(post_jobs=True, jobbot_status=False, current_date=None):
    if jobbot_status:
        sign1 = f"\n----- ** {datetime.now().strftime('%Y-%b-%d')} ** -----"
        sign2 = f"\nMet-Jobs"
        message_text = f" *{'Job_Bot Status: Active'}* {sign1}{sign2}"
        postbot(bot_token, message_text)
    
    if post_jobs:
        if current_date is None:
            # Set the time zone to Hawaii Standard Time (HST)
            hawaii_timezone = pytz.timezone('Pacific/Honolulu')
            current_date = datetime.now(hawaii_timezone).strftime("%Y-%m-%d")
        else:
            current_date = datetime.strptime(current_date, "%Y-%m-%d")
        # Replace this URL with the URL of the RSS feed you want to scrape
        rss_url = "https://maillists.reading.ac.uk/scripts/wa-READING.exe?RSS&L=MET-JOBS&v=2.0&LIMIT=50"

        # Define headers for your requests to mimic a web browser
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9,ar;q=0.8',
            'Cache-Control': 'max-age=0',
            'Referer': 'http://localhost:8888/',
            'Sec-Ch-Ua': '"Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"',
            'Sec-Ch-Ua-Mobile': '?0',
            'Sec-Ch-Ua-Platform': '"macOS"',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'cross-site',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'authority': 'maillists.reading.ac.uk',
            'method': 'GET',
            'path': '/scripts/wa-READING.exe?A2=MET-JOBS;45592065.2310D',
            'scheme': 'https',
            'Cookie': 'WALOGIN=RESET; WALOGIN=RESET',
        }

        # Parse the RSS feed
        feed = feedparser.parse(rss_url)

        # Extract and print the information for each item
        for item in feed.entries:
            pub_date = item.published
            author = item.author
            title = item.title
            guid = item.id
            description = item.description
            # print(pub_date)
            # print(pub_date[:24])
            # Convert the pub_date string to a datetime object
            # pub_date_datetime = datetime.strptime(pub_date[:24], "%a, %d %b %Y %H:%M:%S")
            date_format = "%a, %d %b %Y %H:%M:%S %z"
            pub_date_datetime = datetime.strptime(pub_date, date_format)
            # print(pub_date_datetime)

            # Format the datetime object to get the desired date string
            formatted_date_str = pub_date_datetime.strftime('%Y-%m-%d')
            formatted_date_str = datetime.strptime(formatted_date_str, "%Y-%m-%d")
            if formatted_date_str == current_date:
                # print("--------------------")
                # print(formatted_date_str, current_date)
                # Define keywords to search for
                keywords = ["phd", "ph.d.", " doctoral", "ms ", "m.s.", "master", "assistantship", "fellowship"]

                # Initialize a list to store the links containing the keywords
                filtered_links = []

                for keyword in keywords:
                    if re.search(keyword, title, re.I):
                        filtered_links.append(guid)

        #         print(len(filtered_links))
                # Print the filtered links
                # ...
                for link in filtered_links:
    #                 print(link)
    #                 print(current_date)
    #                 print(title)
    #                 print("----------------------")
                    
                    sign1 = f"\n----- ** {current_date.strftime('%Y-%b-%d')} ** -----"
                    sign2 = f"\n--------------------"
                    link1 = f"\n{link}"
                    message_text = f" *{title}* \n\n{link1}{sign1}{sign2}"
                    # postbot(bot_token, message_text)
                    print(message_text)

# if __name__ == "__main__":
    # metjobs()

In [44]:
metjobs(current_date='2023-11-08')

 *Uppsala University, OX2 PhD research* 


https://maillists.reading.ac.uk/scripts/wa-READING.exe?A2=MET-JOBS;d18b8226.2311B
----- ** 2023-Nov-08 ** -----
--------------------
 *SNSF Ambizione BioPSI PhD student open position* 


https://maillists.reading.ac.uk/scripts/wa-READING.exe?A2=MET-JOBS;3b101c9f.2311B
----- ** 2023-Nov-08 ** -----
--------------------


In [2]:
from es_jobs_net import esjobs

In [3]:
esjobs(current_date_hawaii='2023-11-14')

2023-November
 *Noaa Oar Deputy Assistant Administrator For Programs And Administration, Silver Spring Maryland* 

The announcement for a *Deputy Assistant Administrator for Programs and
Administration *position in the  *Office of Oceanic and Atmospheric
Research (OAR) *has been posted. The announcement will be open for *36*
* days,* from *11/13/23 *to* 12/18/23*. If you are interested in applying
for the position, go to USA JOBS website or click on the announcement link
below.
https://www.usajobs.gov/job/760077900#shorten-link

Megan L. Melamed, PhD (*she, her*)
Deputy Director
NOAA Chemical Sciences Laboratory
Boulder, CO, USA
303.727.0952 (Mobile)
megan.melamed at noaa.gov
csl.noaa.gov/staff/megan.melamed/
<http://www.esrl.noaa.gov/csl/staff/megan.melamed/>

----- ** 2023-11-14 ** -----

 *Postdoctoral Position: Training Machine Learned Parameterizations Of Subgrid Scale Processes Using Ocean Data Assimilation, Princeton, Nj* 

The Atmospheric and Oceanic Sciences Program at Princet

In [30]:
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import os
import re
import pytz
from job_bot import postbot
from api_token import api_key

# def init_bot():
bot_token = api_key()

def esjobs(post_jobs=True, jobbot_status=False, current_date=None):
    if jobbot_status:
        sign1 = f"\n----- ** {datetime.now().strftime('%Y-%b-%d')} ** -----"
        sign2 = f"\nEarth Science Jobs"
        message_text = f" *{'Job_Bot Status: Active'}* {sign1}{sign2}"
        postbot(bot_token, message_text)
    
    if post_jobs:
        if current_date is None:
            # Get the current time in the Hawaii timezone
            hawaii_timezone = pytz.timezone('Pacific/Honolulu')
            current_date_hawaii = datetime.now(hawaii_timezone)    
            current_date = current_date_hawaii
            current_time = current_date.strftime("%B %Y")
        else:
            current_date = datetime.strptime(current_date, "%Y-%m-%d")
            current_time = current_date.strftime("%B %Y")
        # print(current_time)
        url = "https://mailman.ucar.edu/pipermail/es_jobs_net/"

        # Define user-agent and headers
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; \
            x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,\
            application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "en-US,en;q=0.9,ar;q=0.8",
            "Cache-Control": "max-age=0",
            "Connection": "keep-alive",
            "Host": "mailman.ucar.edu",
            "If-Modified-Since": "Thu, 19 Oct 2023 09:12:44 GMT",
            "If-None-Match": "\"1a8c-6080e289aa9c3\"",
            "Sec-Ch-Ua": "\"Chromium\";v=\"118\", \"Google Chrome\";v=\"118\", \"Not=A?Brand\";v=\"99\"",
            "Sec-Ch-Ua-Mobile": "?0",
            "Sec-Ch-Ua-Platform": "\"macOS\"",
            "Sec-Fetch-Dest": "document",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-Site": "none",
            "Sec-Fetch-User": "?1",
            "Upgrade-Insecure-Requests": "1"
        }

        # Send a request to the URL
        response = requests.get(url, headers=headers)
        page_content = response.text

        soup = BeautifulSoup(page_content, "html.parser")

        # Find the link to the latest thread (October 2023)
        latest_thread_link = None
        for row in soup.find_all("tr"):
            cells = row.find_all("td")
            if cells and current_time in cells[0].get_text():
                latest_thread_link = row.find("a", href=True)["href"]
                break

        tempPath = latest_thread_link.split("/")[0]
        print(tempPath)

        # Construct the URL for the latest thread
        if latest_thread_link:
            latest_thread_url = f"{url.rstrip('/')}/{latest_thread_link}"


            # Send a request to the latest thread's URL and scrape its content
            response_latest_thread = requests.get(latest_thread_url, headers=headers)
            latest_thread_content = response_latest_thread.text

            # Parse and extract data from the latest thread's content using BeautifulSoup
            latest_thread_soup = BeautifulSoup(latest_thread_content, "html.parser")

            # Define keywords to search for
            keywords = ["phd", "ph.d.", "doctoral", "ms ", "m.s.", "master", "assistantship", "fellowship"]

            # Initialize a list to store the links containing the keywords
            filtered_links = []

            # Loop through the links and check if they contain any of the keywords
            for link in latest_thread_soup.find_all("a", href=True):  # Fixed this line
                for keyword in keywords:
                    if re.search(keyword, link.get_text(), re.I):
                        filtered_links.append(link['href'])

            # Print the filtered links
            for link in filtered_links:
        #             print(f"posting {link}")
                # print(os.path.join(url, tempPath, link))
                full_url = os.path.join(url, tempPath, link)
                # Send a request to the filtered link and scrape its content
                response_filtered_link = requests.get(full_url, headers=headers)
                filtered_link_content = response_filtered_link.text

                # Parse and extract data from the filtered link's content using BeautifulSoup
                filtered_link_soup = BeautifulSoup(filtered_link_content, "html.parser")
                # You can perform additional parsing or extraction here, or simply print the content
                # print("Content of filtered link:", full_url)        
        #         print(filtered_link_soup.get_text())  # Print the text content of the filtered link
    #             print("--------------------------------------\n")
                pub_date = filtered_link_soup.find("i").get_text()
                format_str = '%a %b %d %H:%M:%S %Y'
                date_time_str = pub_date[:19] + pub_date[23:]

                if len(date_time_str) == 24:
                    date_time_ = datetime.strptime(date_time_str, format_str)
                    date_time_str_formatted = date_time_.strftime("%Y-%m-%d")

                    if date_time_.date() == current_date.date():
                        print(date_time_str_formatted, current_date.strftime('%Y-%m-%d'))

                        # Extract the header
                        header = filtered_link_soup.find("h1")  # Assuming the header is within an h2 element
                        if header:
                            header_text = header.get_text().removeprefix('[ES_JOBS_NET] ')
                            # print("Header:")
                            # print(header_text)

                        # Extract the body
                        body = filtered_link_soup.find("pre")  # Adjust class as needed
                        if body:
                            body_text = body.get_text()
                            # print("\nBody:")
                            split_index = body_text.find("-------------- next part --------------")
                            if split_index != -1:
                            # Keep only the part before "-------------- next part --------------"
                                body_text = body_text[:split_index]
                            # print(body_text)
                        sign1 = f"\n----- ** {current_date} ** -----"
                        sign2 = f"\n"
                        message_text = f" *{header_text.title()}* \n\n{body_text}{sign1}{sign2}"
                        # postbot(bot_token, message_text)
                        print(message_text)

if __name__ == "__main__":
    esjobs(current_date='2023-11-12')

2023-November


AttributeError: 'str' object has no attribute 'strftime'